In [ ]:
# Normalize vectors
import tensorflow as tf
tf.reset_default_graph();
entity_embeddings = tf.get_variable('entity_embeddings', shape=[5,2], initializer=tf.contrib.layers.xavier_initializer())
entity_norms = entity_embeddings/tf.norm(entity_embeddings,axis=-1,keepdims=True)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer());
    a,b = sess.run([entity_norms,entity_embeddings])

In [ ]:
# Look for top closest vectors for each vector through cosine similarity
tf.reset_default_graph();
set1_raw = tf.get_variable('set1_raw', shape=[5,3], initializer=tf.contrib.layers.xavier_initializer())
set2_raw = tf.get_variable('set2_raw', shape=[10,3], initializer=tf.contrib.layers.xavier_initializer())
set1 = set1_raw/tf.norm(set1_raw,axis=-1,keepdims=True);
set2 = set2_raw/tf.norm(set2_raw,axis=-1,keepdims=True);
dist_matrix = tf.matmul(set1,set2,adjoint_b=True)
values, indices = tf.nn.top_k(dist_matrix,k=5);
with tf.Session() as sess:
    tf.global_variables_initializer().run();
    a, b, c, d, e = sess.run([values, indices, set1, set2, dist_matrix])

In [ ]:
# Repeat Tensors
tf.reset_default_graph();
set1_raw = tf.get_variable('set1_raw', shape=[128,1,100], initializer=tf.contrib.layers.xavier_initializer())
set2 = tf.tile(set1_raw,[1,50,1])

In [ ]:
# Pairwise distances between 2D matrices: both L1 and L2 pairwise norms
#==============================================================================
# X: [X1,X2,X3,...,Xn], Y=[Y1,Y2,Y3,...,Ym]
# Z: shape[nXm]: ||X1-Y1|| ||X1-Y2|| ..... ||X1-Ym||
#                ||X2-Y1|| ||X2-Y2|| ..... ||X2-Ym||
#                ......
#                ||Xn-Y1|| ||Xn-Y2|| ..... ||Xn-Ym||
# ||Xi-Yj||=||Xi||+||Yj||-2*dot_prod(Xi,Yj)
#==============================================================================

In [ ]:
import tensorflow as tf
tf.reset_default_graph();
def L1_L2_distances_v2(mat1, mat2, abs_power, name):
    """
    mat1 and mat2 MUST be 2D tensors with the last dimension equal
    Example: mat1.shape=[batch_size,latent_dim], vec2.shape=[n_entitites,latent_dim]
    """
    assert len(mat1.get_shape().as_list())==2
    assert len(mat2.get_shape().as_list())==2
    assert mat1.shape[-1]==mat1.shape[-1]
    # Set final array n_rows and n_cols
    n_rows = mat1.get_shape().as_list()[0];
    n_cols = mat2.get_shape().as_list()[0];
    if abs_power==2: # L2 norm
        mat1_square = tf.pow(tf.reduce_sum(tf.pow(mat1,2),axis=-1,keepdims=True),1/2)
        mat1_square = tf.tile(mat1_square,[1,n_cols])
        mat2_square = tf.pow(tf.reduce_sum(tf.pow(mat2,2),axis=-1,keepdims=True),1/2)
        mat2_square = tf.tile(mat2_square,[1,n_rows])
        mat = mat1_square-2*tf.matmul(mat1,mat2,adjoint_b=True)+tf.transpose(mat2_square);
    elif abs_power==1: #L1 norm
        mat1_tiled = tf.tile(tf.expand_dims(mat1,axis=1), [1,n_cols,1])
        mat2_tiled = tf.tile(tf.expand_dims(mat2,axis=0), [n_rows,1,1])
        mat = tf.reduce_sum(tf.abs(mat1_tiled-mat2_tiled),axis=-1)
    else:
        raise Exception('Invalid configure.abs_power set. Acceptable int values: {1, 2}')
    return mat

swag_graph = tf.Graph();
with swag_graph.as_default():
    mat1 = tf.get_variable('mat1', shape=[10,5], initializer=tf.contrib.layers.xavier_initializer());
    mat2 = tf.get_variable('mat2', shape=[25,5], initializer=tf.contrib.layers.xavier_initializer());
    compute_L1 = L1_L2_distances_v2(mat1, mat2, 1, 'L1')
    compute_L2 = L1_L2_distances_v2(mat1, mat2, 2, 'L2')
    _, top_k_L1 = tf.nn.top_k(compute_L1,k=3)
    _, top_k_L2 = tf.nn.top_k(compute_L2,k=3)
    L2_sorted = tf.contrib.framework.sort(compute_L2, axis=-1, direction='ASCENDING', name='sorted_L2')
with tf.Session(graph=swag_graph) as sess:
    tf.global_variables_initializer().run();
    mat_L1, mat_L2, topK_L1, topK_L2, sorted_L2 = sess.run([compute_L1,compute_L2,top_k_L1,top_k_L2,L2_sorted]);

In [ ]:
# Compute number of similar items between two tensors
import tensorflow as tf
tf.reset_default_graph();
e_graph = tf.Graph();
with e_graph.as_default():
    mat1 = tf.Variable([[1],[2],[5]], name='mat1');
    mat2 = tf.Variable([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]], name='mat2');
    bool_mat = tf.equal(mat1,mat2);
    ranks = tf.argmax(tf.cast(bool_mat,tf.int32), axis=-1);
    mean_rank = tf.reduce_mean(tf.cast(ranks,tf.float32));
    n_equal_in_total = tf.reduce_sum(tf.cast(bool_mat,tf.int32))/tf.size(bool_mat)
    mr_booly = tf.reduce_mean(tf.cast(tf.less_equal(mat1,2),tf.float32));
with tf.Session(graph=e_graph) as sess:
    tf.global_variables_initializer().run();
    mat1, mat2, bool_mat, ranks, mean_rank, n_equal_in_total, mr_booly = \
        sess.run([mat1, mat2, bool_mat, ranks, mean_rank, n_equal_in_total, mr_booly]);